In [1]:
from multi_omics_integration.func import *
from multi_omics_integration.classification_pipeline import *
from multi_omics_integration.processing import *
from multi_omics_integration.evaluation import *
import warnings
from pandas.errors import SettingWithCopyWarning
from sklearn.exceptions import ConvergenceWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=ConvergenceWarning)

# BRCA

In [174]:
# Load the .cct file
data = pd.read_csv('../data/breast/Clinical_Raw.txt', sep='\t', header=None)
data = data.T
data.columns = data.iloc[0]
data = data[1:]

target = pd.DataFrame()
target["SubjectID"] = data['attrib_name']
target["PAM50"] = data['PAM50']
target.dropna(inplace=True)
target.to_csv('../data/breast/Clinical.csv', index=False)
target.set_index('SubjectID', inplace=True)
target

,PAM50
SubjectID,
TCGA.A1.A0SP,Basal
TCGA.A2.A04V,LumA
TCGA.A2.A04Y,LumB
TCGA.A2.A0CQ,LumA
TCGA.A2.A1G4,LumB
...,...
TCGA.LQ.A4E4,LumA
TCGA.OL.A66H,LumA
TCGA.OL.A66I,Basal


In [175]:
mirna = pd.read_csv('../data/breast/miRNASeq_Raw.txt', sep='\t', header=None)
mirna = mirna.T
mirna.columns = mirna.iloc[0]
mirna = mirna[1:]
mirna = mirna.set_index(mirna.columns[0])
mirna = mirna.rename_axis('SubjectID')
mirna

,hsa-let-7a-1,hsa-let-7a-2,hsa-let-7a-3,hsa-let-7b,hsa-let-7c,hsa-let-7d,hsa-let-7e,hsa-let-7f-1,hsa-let-7f-2,hsa-let-7g,...,hsa-mir-940,hsa-mir-941-1,hsa-mir-942,hsa-mir-943,hsa-mir-944,hsa-mir-95,hsa-mir-96,hsa-mir-98,hsa-mir-99a,hsa-mir-99b
SubjectID,,,,,,,,,,,,,,,,,,,,,
TCGA.3C.AAAU,13.1299,14.118,13.1479,14.5952,8.4191,8.6695,10.5228,3.9742,11.8252,8.6015,...,2.8825,0,2.7072,0.2479,1.6146,2.3574,6.9187,5.7812,7.0356,15.5065
TCGA.3C.AALI,12.9183,13.9224,12.9134,14.5127,9.6483,9.0065,9.1343,4.4543,12.6791,8.4592,...,1.0712,0,3.1985,0,0.7042,1.1753,5.9366,6.4437,7.8914,13.6263
TCGA.3C.AALJ,13.0122,14.0101,13.0287,13.4197,9.3147,9.2793,11.3962,5.3505,13.5304,9.233,...,1.2114,0,2.2073,0,0.8217,1.4031,6.6184,6.8905,7.5882,15.0139
TCGA.3C.AALK,13.1449,14.1418,13.1514,14.6673,11.5119,8.3891,10.3701,4.2378,12.6528,8.4756,...,0.9625,0,2.163,0,2.163,1.4538,6.2011,5.4122,10.033,14.5548
TCGA.4H.AAAK,13.4118,14.4136,13.4206,14.4386,11.6944,8.4579,10.7422,4.5572,13.0097,8.3855,...,0.4053,0,1.6361,0,1.0944,0.8567,4.3887,5.1449,10.0795,14.6504
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA.WT.AB44,13.3759,14.3667,13.37,14.5141,11.9267,8.5241,10.5369,4.609,13.3679,9.1616,...,0.8573,0,1.1489,0,0.4914,1.1489,6.3365,5.8735,10.1058,13.9961
TCGA.XX.A899,14.0362,15.0364,14.0434,14.3396,12.362,7.663,11.0863,5.2437,14.1539,9.767,...,0.5887,0,2.6649,0,2.0442,1.7304,4.14,5.8666,10.9073,13.7815
TCGA.XX.A89A,13.6797,14.6849,13.6916,14.1983,12.6844,8.5569,11.3152,4.8829,14.0395,9.6062,...,1.4205,0,1.5176,0,0.6406,0.9498,5.4236,5.787,10.3566,14.0595


In [176]:
rna = pd.read_csv('../data/breast/RNASeq_Raw.cct', sep='\t', header=None)
rna = rna.T
rna.columns = rna.iloc[0]
rna = rna[1:]
rna = rna.set_index(rna.columns[0])
rna = rna.rename_axis('SubjectID')
rna

/var/folders/bl/4vcbfl1n40sbx41l9qnkmm3h0000gn/T/ipykernel_56202/4281338175.py:1: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251

,A1BG,A1CF,A2BP1,A2LD1,A2ML1,A2M,A4GALT,A4GNT,AAA1,AAAS,...,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,psiTPTE22,tAKR
SubjectID,,,,,,,,,,,,,,,,,,,,,
TCGA.3C.AAAU,7.63,0,0,6.6999,1.2501,12.5017,6.1136,3.2655,0.4273,9.345,...,7.0289,9.9784,10.6965,8.0196,10.24,11.7765,10.8887,10.2064,1.4454,0.0
TCGA.3C.AALI,7.8971,0,0,6.1672,2.4196,12.8866,7.3101,0.6265,0,9.796,...,5.9264,8.8125,10.3924,7.6401,9.2398,12.4267,10.3659,8.6715,9.8573,0.0
TCGA.3C.AALJ,8.7287,0.931,0,7.3422,0,13.1101,9.1671,0,0,9.5008,...,5.1842,9.0634,9.5884,8.3786,9.0585,12.4146,9.8825,8.9958,5.1842,0.0
TCGA.3C.AALK,7.5851,0,0,5.9888,1.4087,13.4201,8.987,0,0,9.6036,...,5.808,8.7772,9.7564,7.4629,9.2488,12.4748,9.6113,9.4551,6.0792,0.0
TCGA.4H.AAAK,8.0762,0.5115,2.272,7.2796,2.1389,13.2268,8.4226,0.5115,0.5115,9.6797,...,5.642,8.7322,10.0373,3.911,9.6013,11.9811,9.702,9.7858,7.5564,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA.WT.AB44,8.8835,0,0,5.9711,2.91,12.4015,8.4828,0.8596,0,10.3078,...,4.6687,7.9339,9.5961,6.6374,8.3047,13.4171,9.6468,8.7947,9.5036,0.0
TCGA.XX.A899,7.8088,0,0.4789,7.0469,0.4789,14.3127,8.9845,0.8379,0,9.4678,...,6.1672,9.3325,10.3078,4.4764,9.4288,12.3217,10.918,9.3903,6.6355,0.0
TCGA.XX.A89A,8.0008,1.7492,1.2733,6.3386,5.8263,14.0614,9.2679,1.9389,0.9595,9.8182,...,5.6338,8.42,10.4268,7.3697,9.5465,12.4196,10.4903,9.8989,7.8841,0.9595


In [177]:
meth = pd.read_csv('../data/breast/Methylation_Raw.cct', sep='\t', header=None)
meth = meth.T
meth.columns = meth.iloc[0]
meth = meth[1:]
meth = meth.set_index(meth.columns[0])
meth = meth.rename_axis('SubjectID')
meth

/var/folders/bl/4vcbfl1n40sbx41l9qnkmm3h0000gn/T/ipykernel_56202/566015772.py:1: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,

,RBL2,VDAC3,ACTN1,ATP2A1,SFRP1,NIPA2,MAN1B1,TSEN34,LRRC16A,CNBP,...,BAGE5,BAGE,BAGE2,BAGE4,BAGE3,MIR637,LOC100130932,GOLGA8F,GOLGA8G,MIR7-1
SubjectID,,,,,,,,,,,,,,,,,,,,,
TCGA.3C.AAAU,-0.4444,-0.4548,0.231,0.3568,-0.4737,-0.4817,0.4142,-0.4366,-0.461,-0.4151,...,-0.042,-0.042,-0.042,-0.042,-0.042,0.342,0.2383,-0.1168,-0.1168,NaN
TCGA.3C.AALI,-0.4313,-0.4692,0.2859,0.3972,-0.2806,-0.474,0.4373,-0.4527,-0.4643,0.2093,...,-0.064,-0.064,-0.064,-0.064,-0.064,0.3857,0.3783,-0.1271,-0.1271,NaN
TCGA.3C.AALJ,-0.4285,-0.4697,0.2664,0.3482,-0.0637,-0.4783,0.3738,-0.443,-0.4416,-0.4185,...,-0.0873,-0.0873,-0.0873,-0.0873,-0.0873,0.3319,0.1929,0.0066,0.0066,NaN
TCGA.3C.AALK,-0.4043,-0.4714,0.2174,0.3533,-0.044,-0.4827,0.3704,-0.443,-0.4558,-0.3999,...,0.1426,0.1426,0.1426,0.1426,0.1426,0.3457,0.2445,-0.0071,-0.0071,NaN
TCGA.4H.AAAK,-0.4083,-0.4651,0.1717,0.3406,-0.088,-0.4788,0.4143,-0.4442,-0.4675,-0.3977,...,0.0218,0.0218,0.0218,0.0218,0.0218,0.3542,0.3128,-0.0929,-0.0929,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA.WT.AB44,-0.3903,-0.4683,0.2731,0.3682,-0.1018,-0.4842,0.3842,-0.4408,-0.4601,-0.3462,...,0.2111,0.2111,0.2111,0.2111,0.2111,0.3322,0.2746,-0.0284,-0.0284,NaN
TCGA.XX.A899,-0.45,-0.4721,0.2597,0.4152,0.0244,-0.4843,0.4211,-0.4391,-0.4622,-0.4471,...,0.1998,0.1998,0.1998,0.1998,0.1998,0.3908,0.4161,-0.0702,-0.0702,-0.2598
TCGA.XX.A89A,-0.4237,-0.4649,0.1951,0.3586,-0.4071,-0.4862,0.4494,-0.4609,-0.4639,-0.4376,...,0.1236,0.1236,0.1236,0.1236,0.1236,0.4054,0.4089,-0.121,-0.121,-0.247


In [178]:
data = {'mirna': mirna, 'rna': rna, 'meth': meth}  # List of dataframes
new_data = {}

for modality in data:
    new_data[modality] = data[modality].loc[~data[modality].index.duplicated(keep='first')]

In [179]:
suffixed = {}

# Add suffixes to each DataFrame
for key in new_data.keys():
    # Add suffix
    suffixed[key] = new_data[key].add_suffix("_" + key)

# Merge datasets based on index and aligning them with the target

merged = pd.concat(
        [pd.concat(suffixed.values(), axis=1, join="inner"), target], axis=1, join="inner"
)

y = merged['PAM50']

merged_X = (merged.drop(columns='PAM50')).astype("float")

X = {}

for key in new_data.keys():
    # Extract the corresponding columns for each dataset
    X[key] = (merged.loc[:, suffixed[key].columns]).astype("float")

In [180]:
X['mirna'].to_csv('../data/breast/miRNASeq.csv')
X['rna'].to_csv('../data/breast/RNASeq.csv')
X['meth'].to_csv('../data/breast/Methylation.csv')
y.to_csv('../data/breast/Clinical.csv')

In [120]:
y.value_counts()

LumA     229
LumB     100
Basal     71
Her2      30
Name: PAM50, dtype: int64

# KIPAN

In [8]:
# Load the .cct file
data = pd.read_csv('../data/kipan/Clinical_Raw.txt', sep='\t', header=None)
data = data.T
data.columns = data.iloc[0]
data = data[1:]

target = pd.DataFrame()
target["SubjectID"] = data['attrib_name']
target["histological_type"] = data['histological_type']
target.dropna(inplace=True)
target.to_csv('../data/kipan/Clinical.csv', index=False)
target.set_index('SubjectID', inplace=True)
target

,histological_type
SubjectID,
TCGA.KL.8328,kidneychromophobe
TCGA.KL.8339,kidneychromophobe
TCGA.KM.8439,kidneychromophobe
TCGA.KM.8441,kidneychromophobe
TCGA.KM.8442,kidneychromophobe
...,...
TCGA.Y8.A896,kidneypapillaryrenalcellcarcinoma
TCGA.Y8.A897,kidneypapillaryrenalcellcarcinoma
TCGA.Y8.A8RY,kidneypapillaryrenalcellcarcinoma


In [2]:
datasets = {
            'rna': '../data/kipan/RNASeq_Raw.cct',
            'cnv': '../data/kipan/CNV_Raw.cct',
            'RPPA': '../data/kipan/RPPA_Raw.txt',
            'meth': '../data/kipan/Methylation_Raw.cct',
            'mirna': '../data/kipan/miRNASeq_Raw.txt'
}

In [15]:
data = {}

for modality in datasets:
    data[modality] = pd.read_csv(datasets[modality], sep='\t', header=None).T
    data[modality].columns = data[modality].iloc[0]
    data[modality] = data[modality][1:]
    data[modality] = data[modality].set_index(data[modality].columns[0])
    data[modality] = data[modality].rename_axis('SubjectID')

/var/folders/bl/4vcbfl1n40sbx41l9qnkmm3h0000gn/T/ipykernel_790/3240453431.py:4: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,2

In [16]:
for modality in datasets:
    print(modality, data[modality].shape)

rna (889, 20190)
cnv (882, 24776)
RPPA (757, 166)
meth (660, 20116)
mirna (611, 799)


In [4]:
new_data = {}

for modality in data:
    new_data[modality] = data[modality].loc[~data[modality].index.duplicated(keep='first')]

In [9]:
suffixed = {}

# Add suffixes to each DataFrame
for key in new_data.keys():
    # Add suffix
    suffixed[key] = new_data[key].add_suffix("_" + key)

# Merge datasets based on index and aligning them with the target

merged = pd.concat(
        [pd.concat(suffixed.values(), axis=1, join="inner"), target], axis=1, join="inner"
)

y = merged['histological_type']

merged_X = (merged.drop(columns='histological_type')).astype("float")

X = {}

for key in new_data.keys():
    # Extract the corresponding columns for each dataset
    X[key] = (merged.loc[:, suffixed[key].columns]).astype("float")

In [11]:
X['cnv'].to_csv('../data/kipan/CNV.csv')
X['rna'].to_csv('../data/kipan/RNASeq.csv')
X['RPPA'].to_csv('../data/kipan/RPPA.csv')
X['meth'].to_csv('../data/kipan/Methylation.csv')
X['mirna'].to_csv('../data/kipan/miRNASeq.csv')
y.to_csv('../data/kipan/Clinical.csv')